## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weatherpy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Naze,JP,28.3667,129.4833,59.92,64,99,19.66,overcast clouds
1,1,College,US,64.8569,-147.8028,-27.45,67,1,0.00,clear sky
2,2,Buchanan,LR,5.8808,-10.0467,82.72,71,75,3.04,broken clouds
3,3,Araripe,BR,-7.2125,-40.0461,69.33,88,100,7.43,overcast clouds
4,4,Rikitea,PF,-23.1203,-134.9692,75.96,75,100,7.96,moderate rain


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Buchanan,LR,5.8808,-10.0467,82.72,71,75,3.04,broken clouds
4,4,Rikitea,PF,-23.1203,-134.9692,75.96,75,100,7.96,moderate rain
7,7,Esperance,AU,-33.8667,121.9000,78.64,64,0,21.14,clear sky
16,16,Atuona,PF,-9.8000,-139.0333,78.12,77,17,12.33,few clouds
20,20,Saldanha,ZA,-33.0117,17.9442,80.65,69,12,11.54,few clouds
23,23,Namibe,AO,-15.1961,12.1522,77.63,69,0,0.00,clear sky
28,28,Cape Town,ZA,-33.9258,18.4232,83.12,50,0,8.05,clear sky
33,33,Vaini,TO,-21.2000,-175.2000,78.96,100,75,3.44,broken clouds
37,37,Geraldton,AU,-28.7667,114.6000,81.01,61,0,20.71,clear sky
39,39,Busselton,AU,-33.6500,115.3333,88.09,54,0,23.89,clear sky


In [9]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.count()

City_ID                187
City                   187
Country                184
Lat                    187
Lng                    187
Max Temp               187
Humidity               187
Cloudiness             187
Wind Speed             187
Current Description    187
dtype: int64

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                184
City                   184
Country                184
Lat                    184
Lng                    184
Max Temp               184
Humidity               184
Cloudiness             184
Wind Speed             184
Current Description    184
dtype: int64

In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Buchanan,LR,82.72,broken clouds,5.8808,-10.0467,
4,Rikitea,PF,75.96,moderate rain,-23.1203,-134.9692,
7,Esperance,AU,78.64,clear sky,-33.8667,121.9000,
16,Atuona,PF,78.12,few clouds,-9.8000,-139.0333,
20,Saldanha,ZA,80.65,few clouds,-33.0117,17.9442,
23,Namibe,AO,77.63,clear sky,-15.1961,12.1522,
28,Cape Town,ZA,83.12,clear sky,-33.9258,18.4232,
33,Vaini,TO,78.96,broken clouds,-21.2000,-175.2000,
37,Geraldton,AU,81.01,clear sky,-28.7667,114.6000,
39,Busselton,AU,88.09,clear sky,-33.6500,115.3333,


In [16]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")  

IndexError: list index out of range

In [19]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_df.count()

City_ID                184
City                   184
Country                184
Lat                    184
Lng                    184
Max Temp               184
Humidity               184
Cloudiness             184
Wind Speed             184
Current Description    184
dtype: int64

In [25]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Buchanan,LR,82.72,broken clouds,5.8808,-10.0467,Kamaneahn Hotel Inc
4,Rikitea,PF,75.96,moderate rain,-23.1203,-134.9692,People ThankYou
7,Esperance,AU,78.64,clear sky,-33.8667,121.9000,Hospitality Esperance
16,Atuona,PF,78.12,few clouds,-9.8000,-139.0333,Villa Enata
20,Saldanha,ZA,80.65,few clouds,-33.0117,17.9442,Blue Bay Lodge
23,Namibe,AO,77.63,clear sky,-15.1961,12.1522,Hotel Chik Chik Namibe
28,Cape Town,ZA,83.12,clear sky,-33.9258,18.4232,Southern Sun Waterfront Cape Town
33,Vaini,TO,78.96,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
37,Geraldton,AU,81.01,clear sky,-28.7667,114.6000,Broadwater Mariner Resort
39,Busselton,AU,88.09,clear sky,-33.6500,115.3333,Observatory Guest House


In [26]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [37]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} and <dd>{Max Temp} °F</dd><dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [38]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))